In [ ]:
# Install dependencies
!pip install -q amplpy ampltools

In [ ]:
# Google Colab & AMPL integration
MODULES, LICENSE_UUID = ["coin", 'gurobi', "cplex", "highs", "gokestrel"], "42fc7eb6-69aa-445d-b655-3ad24d836541"
from amplpy import tools
from ampltools import cloud_platform_name, ampl_notebook, register_magics

# instantiate AMPL object and register magics
if cloud_platform_name() is None:
    ampl = AMPL() # Use local installation of AMPL
else:
    ampl = tools.ampl_notebook(modules=MODULES, license_uuid=LICENSE_UUID, g=globals())

register_magics(ampl_object=ampl)

Licensed to Bundle #6741.7193 expiring 20241231: INFO 645 Prescriptive Analytics, Prof. Paul Brooks, Virginia Commonwealth University.


In [ ]:
ampl.eval('''
reset;

# Define sets
set FundTypes;  # Set of different types of funds

# Define parameters
param maxAlloc {FundTypes};  # Maximum allocation in each fund
param rateOfReturn {FundTypes};  # Return rate for each fund
param riskFactor {FundTypes};  # Risk factor per $1000 invested for each fund
param totalLimit;  # Total investment limit across all funds
param maxRiskLevel;  # Maximum acceptable risk for the portfolio

# Define decision variables
var Allocation {FundTypes} >= 0;  # Allocation amount in each fund type

# Objective function: Maximize total return
maximize PortfolioReturn:
    sum {f in FundTypes} rateOfReturn[f] * Allocation[f];

# Constraints
subject to BudgetConstraint:
    sum {f in FundTypes} Allocation[f] <= totalLimit;

subject to FundLimits {f in FundTypes}:
    Allocation[f] <= maxAlloc[f];

subject to RiskTolerance:
    sum {f in FundTypes} (riskFactor[f] * (Allocation[f] / 1000)) <= maxRiskLevel;
''')


In [ ]:
# Define the set of funds
ampl.set['FundTypes'] = ['Internet', 'BlueChip']

# Parameters for maximum investment limits, return rates, and risk ratings
ampl.param['maxAlloc'] = {'Internet': 35000, 'BlueChip': 50000}
ampl.param['rateOfReturn'] = {'Internet': 0.12, 'BlueChip': 0.09}
ampl.param['riskFactor'] = {'Internet': 6, 'BlueChip': 4}

# Overall investment and risk tolerance limits
ampl.param['totalLimit'] = 50000
ampl.param['maxRiskLevel'] = 240
ampl.eval('''expand;''')


maximize PortfolioReturn:
	0.12*Allocation['Internet'] + 0.09*Allocation['BlueChip'];

subject to BudgetConstraint:
	Allocation['Internet'] + Allocation['BlueChip'] <= 50000;

subject to FundLimits['Internet']:
	Allocation['Internet'] <= 35000;

subject to FundLimits['BlueChip']:
	Allocation['BlueChip'] <= 50000;

subject to RiskTolerance:
	0.006*Allocation['Internet'] + 0.004*Allocation['BlueChip'] <= 240;



In [ ]:
ampl.setOption('solver', 'cbc')
ampl.solve()

cbc 2.10.10: cbc 2.10.10: optimal solution; objective 5100
0 simplex iterations


In [ ]:
# Print results
# Assuming 'PortfolioReturn' is the name of the objective function and 'Allocation' is the decision variable array

# First, solve the model
ampl.solve()

# Retrieve and print the total return objective value
obj = ampl.getObjective('PortfolioReturn')
print("\nTotal expected annual return is: $", obj.value(), "\n")

# Print the allocation amounts in each fund vertically
print("Investment amounts :")
ampl.display('Allocation')


cbc 2.10.10: cbc 2.10.10: optimal solution; objective 5100
0 simplex iterations

Total expected annual return is: $ 5100.0 

Investment amounts :
Allocation [*] :=
BlueChip  30000
Internet  20000
;

